In [1]:
%cd

import os
# 檔案的 URL 和目標路徑
url = "https://download.openmmlab.com/mmsegmentation/v0.5/pspnet/pspnet_r50-d8_512x1024_40k_cityscapes/pspnet_r50-d8_512x1024_40k_cityscapes_20200605_003338-2966598c.pth"
target_dir = "./cv_final/mmsegmentation/checkpoints"
filename = "pspnet_r50-d8_512x1024_40k_cityscapes_20200605_003338-2966598c.pth"

# 檢查檔案是否已存在
if not os.path.exists(os.path.join(target_dir, filename)):
    # 如果不存在，則進行下載
    %cd ./cv_final/mmsegmentation/checkpoints
    !wget https://download.openmmlab.com/mmsegmentation/v0.5/pspnet/pspnet_r50-d8_512x1024_40k_cityscapes/pspnet_r50-d8_512x1024_40k_cityscapes_20200605_003338-2966598c.pth -P ../checkpoints
    %cd
    print(f"\n下載完成: {filename}")
else:
    print(f"檔案已存在: {filename}")

/home/leowang707
檔案已存在: pspnet_r50-d8_512x1024_40k_cityscapes_20200605_003338-2966598c.pth


In [2]:
%cd ./cv_final/mmsegmentation
import numpy as np
import mmcv
import torch
import matplotlib.pyplot as plt
from mmengine.model.utils import revert_sync_batchnorm
from mmseg.apis import init_model, inference_model, show_result_pyplot

/home/leowang707/cv_final/mmsegmentation


In [3]:
import torch
print(torch.__version__)

2.3.0+cu118


In [4]:
config_file = 'configs/pspnet/pspnet_r50-d8_4xb2-40k_cityscapes-512x1024.py'
checkpoint_file = 'checkpoints/pspnet_r50-d8_512x1024_40k_cityscapes_20200605_003338-2966598c.pth'

In [5]:
# build the model from a config file and a checkpoint file
model = init_model(config_file, checkpoint_file, device='cuda:0')

/home/leowang707/cv_final/mmsegmentation/mmseg/models/losses/cross_entropy_loss.py:250: UserWarning: Default ``avg_non_ignore`` is False, if you would like to ignore the certain label and average loss over non-ignore labels, which is the same with PyTorch official cross_entropy, set ``avg_non_ignore=True``.
  warnings.warn(


Loads checkpoint by local backend from path: checkpoints/pspnet_r50-d8_512x1024_40k_cityscapes_20200605_003338-2966598c.pth


In [6]:
%cd
%cd ./cv_final/notebooks
# test a single image
img = "input.png"

if not torch.cuda.is_available():
    model = revert_sync_batchnorm(model)
result = inference_model(model, img)

# show the results
vis_result = show_result_pyplot(model, img, result, show=False)
plt.imshow(vis_result)
plt.axis('off')  # 关闭坐标轴
plt.show()

/home/leowang707
/home/leowang707/cv_final/notebooks


FileNotFoundError: [Errno 2] No such file or directory: 'input.png'

In [ ]:
# Extract the mask
# Access the pred_sem_seg attribute from the result object
mask = result.pred_sem_seg.data[0].cpu().numpy()

# Print out unique class indices in the mask
unique_classes = np.unique(mask)
print("Unique classes in the mask:", unique_classes)

# Ensure the directory exists
save_dir = '../notebooks/SETR_result/image1'
os.makedirs(save_dir, exist_ok=True)

# Define Cityscapes class labels using model.dataset_meta['classes']
# Extract class labels from model.dataset_meta['classes']
class_labels = model.dataset_meta['classes']

# Define Cityscapes class labels using model.dataset_meta['classes']
cityscapes_labels = {class_id: class_labels[class_id] for class_id in unique_classes}

# Save individual masks for each class and create label data
label_data = []
for class_id in unique_classes:
    class_mask = (mask == class_id).astype(np.uint8) * 255  # Convert to 8-bit image
    save_path = os.path.join(save_dir, f'mask_class_{class_id}.png')
    mmcv.imwrite(class_mask, save_path)
    label = cityscapes_labels.get(class_id, 'unknown')
    label_data.append(f"Label ID: {class_id}, Category: {label}")

# Save label data as a text file
labels_file = os.path.join(save_dir, 'labels.txt')
with open(labels_file, 'w') as f:
    for label_info in label_data:
        f.write(f"{label_info}\n")

print(f"Files saved to: {save_dir}")